In [5]:
Pkg.add("MAT")
Pkg.add("GR")
using MAT # This package enables reading .mat files
using JuMP 
using Plots
using Interact
gr()

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of MAT
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Cloning cache of GR from https://github.com/jheinen/GR.jl.git
INFO: Installing GR v0.17.0
INFO: Building GR
INFO: Downloading pre-compiled GR 0.21.0 Darwin binary
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.8M  100 20.8M    0     0  3006k      0  0:00:07  0:00:07 --:--:-- 4516k
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of GR
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Precompiling module GR.


Plots.GRBackend()

In [3]:
file = matopen("ieee_test_system.mat") # open the .mat file with input data
c_g=read(file, "c_g") # incremental costs of generators
c_g0=read(file, "c_g0") # no load costs of generators
g_max=read(file, "g_max") # maximum power outputs of generators
g_min=read(file, "g_min") # minimum power outputs of generators
g_map=read(file, "g_map") # map of generators

d=read(file, "d") # demand

w_f=read(file, "w_f") # wind forecasts for each wind farm
w_map=read(file, "w_map") # map of wind farms

f_map=read(file, "f_map") # map of transmission lines
f_max=read(file, "f_max") #maximum power flow limits of transmission lines 
x=read(file, "x") # reactance of transmission lines
close(file)

N_gen = length(c_g) # number of generators
N_bus = length(d) # number of buses
N_line = length(f_max) # number of lines
N_wind = length(w_f) # number of wind farms


c_w = 10 # dispatch cost of wind generation

LoadError: LoadError: File "ieee_test_system.mat" does not exist and create was not specified
while loading In[3], in expression starting on line 1

In [4]:
f_map

LoadError: LoadError: UndefVarError: f_map not defined
while loading In[4], in expression starting on line 1

In [ ]:
function solve_ed(g_max, g_min, c_g, c_w, d, w_f)
    #Define the economic dispatch (ED) model
    ed = Model() 
    
    # Define decision variables    
    @variable(ed, g_min[i] <= g[i=1:N_gen] <= g_max[i]) # power output of generators
    @variable(ed, 0 <= w  <= w_f ) # wind power injection

    # Define the objective function
    @objective(ed,Min,sum(c_g[i] * g[i] for i=1:N_gen)+ c_w * w)

    # Define the constraint on the wind power injection
    @constraint(ed, w <= w_f)

    # Define the power balance constraint
    @constraint(ed, sum(g[i] for i=1:N_gen) + w == d)

    # Solve statement
    solve(ed)
    # return the optimal value of the objective function and its minimizers
    return getvalue(g), getvalue(w), w_f-getvalue(w), getobjectivevalue(ed)
end